# 1.导入所需要的包

In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split #用来划分训练集与测试集

In [3]:
train = pd.read_csv(r'C:/Users/CHICHI/Desktop/文本正则化/input/en_train.csv')
train.shape

(9918441, 5)

In [2]:
max_num_features = 10 
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 9918441 #训练数据大小

In [3]:
out_path = r'C:/Users/CHICHI/Desktop/output'
df = pd.read_csv(r'C:/Users/CHICHI/Desktop/input/en_train.csv')

In [4]:
x_data = []
y_data =  pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()

35

In [5]:
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

In [6]:
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

In [7]:
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

In [8]:
print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

Total number of samples: 9918441
Use:  9918441
x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1  66 114 105 108 108  97
 110 116  97 105  -1 105 115   0   0   0   0   0   0   0   0  -1]
y_data sample:
0
labels:
Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')


In [9]:
x_train = x_data
y_train = y_data
gc.collect()

40

In [10]:
x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=20817)
"""
#划分测试集与验证集
test_size：测试数据占样品数据的比例
random_state：设置随机数种子，保证每次都是同一个随机数，若为0或者不填，则每次得到的数据都不一样
"""

gc.collect()
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

XGBoost算法参数设置

In [ ]:
param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 
         'num_class':num_class,
         'eval_metric':'merror'}
"""
##常规参数
silent：静默模式，为1时模型运行不输出
nthread [default to maximum number of threads available if not set]：线程数，可以不用设置，原始代码值为-1
，导致cpu未能跑满

##模型参数
eta（学习率）:shrinkage参数，用于更新叶子节点权重，乘以该系数，以防止步长过大。但是参数值若过大，越可能无法收敛，把
eta设置小一点可以使得后面的学习更加仔细
max_depth:给定树的最大深度，默认为3。树越深，越容易过拟合

##学习任务参数
objective（最小化损失函数类型）:给定损失函数，默认为“binary:logistic”，若设置XGBoost使用softmax
函数做多分类任务，需要设置参数num_class（类别个数）
eval_metric（学习目标参数）:可以设置为merror，其为多分类错误率，计算公式为(wrong cases)/(all cases)
"""

In [11]:
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred 
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

model.save_model(os.path.join(out_path, 'xgb_model'))

[18:54:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:54:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	valid-merror:0.00902	train-merror:0.00898
Multiple eval metrics have been passed: 'train-merror' will be used for early stopping.

Will train until train-merror hasn't improved in 20 rounds.
[10]	valid-merror:0.00435	train-merror:0.00426
[20]	valid-merror:0.00343	train-merror:0.00315
[30]	valid-merror:0.00298	train-merror:0.00257
[40]	valid-me